In [ ]:
#EDA

In [1]:
import pandas as pd
path=r"C:\path\KPMG_VI_New_raw_data_update_final.xlsx"
tran=pd.read_excel(path, sheet_name="Transactions", header=1)
newcust=pd.read_excel(path, sheet_name="NewCustomerList", header=1)
cdemo=pd.read_excel(path, sheet_name="CustomerDemographic", header=1)
cadd=pd.read_excel(path, sheet_name="CustomerAddress", header=1)

C:\Users\Pranchali\AppData\Local\Temp\ipykernel_15708\2973115457.py:4: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  newcust=pd.read_excel(path, sheet_name="NewCustomerList", header=1)
C:\Users\Pranchali\AppData\Local\Temp\ipykernel_15708\2973115457.py:5: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  cdemo=pd.read_excel(path, sheet_name="CustomerDemographic", header=1)


In [2]:
print("Transactions ",tran.shape)
print("CustomerDemographic ",cdemo.shape)
print("CustomerAddress ",cadd.shape)

Transactions  (20000, 13)
CustomerDemographic  (4000, 13)
CustomerAddress  (3999, 6)


In [ ]:
cdemo.head()

In [ ]:
#CustomerDemographic dataset cleaning

In [ ]:
cdemo.info()

In [3]:
cdemo.duplicated().sum()

0

In [ ]:
#there are no duplicates in this dataset

In [3]:
#drop default column as it contains irrelevant values
cdemo=cdemo.drop(['default'], axis=1)

In [5]:
#check null values in each column
round((cdemo.isnull().sum())).sort_values(ascending=False)

job_industry_category                  656
job_title                              506
last_name                              125
DOB                                     87
tenure                                  87
customer_id                              0
first_name                               0
gender                                   0
past_3_years_bike_related_purchases      0
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
dtype: int64

In [4]:
#for object columns impute values with n/a
cdemo['job_industry_category']=cdemo['job_industry_category'].fillna("na")
cdemo['job_title']=cdemo['job_title'].fillna("na")
cdemo['last_name']=cdemo['last_name'].fillna("na")

In [ ]:
#as the dataset is for the timeperiod of 2017, age will be calculated as of 2017

In [5]:
cdemo['DOB']=pd.to_datetime(cdemo['DOB'])
current_year=2017
cdemo['Age']=current_year-cdemo['DOB'].dt.year

In [6]:
#impute age and tenure with mode
agemode=int(cdemo['Age'].mode())
tenuremode=int(cdemo['tenure'].mode())
cdemo['Age'].fillna(agemode, inplace=True)
cdemo['tenure'].fillna(tenuremode, inplace=True)
cdemo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          4000 non-null   int64         
 1   first_name                           4000 non-null   object        
 2   last_name                            4000 non-null   object        
 3   gender                               4000 non-null   object        
 4   past_3_years_bike_related_purchases  4000 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            4000 non-null   object        
 7   job_industry_category                4000 non-null   object        
 8   wealth_segment                       4000 non-null   object        
 9   deceased_indicator                   4000 non-null   object        
 10  owns_car    

In [ ]:
cdemo.head()

In [9]:
#normalize gender column
cdemo['gender'].value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [7]:
cdemo['gender']=cdemo['gender'].replace('Female', 'F').replace('Femal', 'F').replace('Male','M')
cdemo['gender'].value_counts()

F    2039
M    1873
U      88
Name: gender, dtype: int64

In [ ]:
#CustomerAddress dataset cleaning

In [ ]:
cadd.head()

In [11]:
#check null values in each column
round((cadd.isnull().sum())).sort_values(ascending=False)

customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64

In [12]:
cadd.duplicated().sum()

0

In [ ]:
cadd.info()

In [ ]:
cadd['state'].value_counts()

In [8]:
#normalizing values in state column
cadd['state']=cadd['state'].replace('New South Wales', 'NSW').replace('Victoria', 'VIC')

In [9]:
cadd['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

In [ ]:
#Transactions dataset cleaning

In [ ]:
tran.head()

In [15]:
tran.duplicated().sum()

0

In [16]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [10]:
#converting excel date values to datetime format
tran['product_first_sold_date']=pd.to_datetime(tran['product_first_sold_date'], unit='D', origin='1899-12-30')

In [18]:
#check null values in each column
round((tran.isnull().sum())).sort_values(ascending=False)

online_order               360
brand                      197
product_line               197
product_class              197
product_size               197
standard_cost              197
product_first_sold_date    197
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
order_status                 0
list_price                   0
dtype: int64

In [11]:
tran[tran.isnull().any(axis=1)]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
97,98,49,333,2017-06-23,NaN,Approved,Trek Bicycles,Road,medium,medium,533.51,400.13,2003-07-21
136,137,0,431,2017-09-23,0.0,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaT
159,160,0,3300,2017-08-27,0.0,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaT
166,167,90,3177,2017-04-26,NaN,Approved,Norco Bicycles,Standard,low,medium,363.01,290.41,2005-05-10
169,170,6,404,2017-10-16,NaN,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2003-08-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19635,19636,98,1389,2017-07-26,NaN,Approved,Trek Bicycles,Standard,high,medium,358.39,215.03,2004-01-16
19793,19794,0,2860,2017-01-13,0.0,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaT
19843,19844,3,8,2017-01-28,NaN,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2003-07-21
19859,19860,0,2468,2017-06-24,1.0,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaT


In [12]:
#dropping null values from transaction dataset
tran.dropna(inplace=True)

In [22]:
tran.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10


In [13]:
print("Transactions ",tran.shape)
print("CustomerDemographic ",cdemo.shape)
print("CustomerAddress ",cadd.shape)

Transactions  (19445, 13)
CustomerDemographic  (4000, 13)
CustomerAddress  (3999, 6)


In [14]:
#unique customer ids in all 3 datasets
print("Transactions ",tran['customer_id'].sort_values().unique())
print("CustomerDemographic ",cdemo['customer_id'].sort_values().unique())
print("CustomerAddress ",cadd['customer_id'].sort_values().unique())

Transactions  [   1    2    3 ... 3499 3500 5034]
CustomerDemographic  [   1    2    3 ... 3998 3999 4000]
CustomerAddress  [   1    2    4 ... 4001 4002 4003]


In [18]:
tran['customer_id'].value_counts().sum()

19445

In [ ]:
#CustomerDemographic has upto 4000 ids, whereas CustomerAddress has extra 3 ids, Transaction has extra id

In [19]:
#merge all 3 datasets, assuming CustomerDemographic as master data
merged=cdemo.merge(cadd,on='customer_id').merge(tran, on='customer_id')

In [20]:
merged.shape

(19416, 30)

In [21]:
merged.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,...,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,2017-12-23,0.0,Approved,OHM Cycles,Standard,medium,medium,235.63,125.07,2005-05-10
1,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,2017-04-06,1.0,Approved,Solex,Standard,medium,medium,1577.53,826.51,2008-03-19
2,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,2017-05-11,1.0,Approved,Trek Bicycles,Road,low,small,1720.70,1531.42,2003-07-21
3,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,2017-01-05,0.0,Approved,Norco Bicycles,Standard,medium,medium,360.40,270.30,2003-09-09
4,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,2017-02-21,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2005-08-09


In [23]:
outpath=r"C:\path\KPMG_VI_merged.xlsx"
merged.to_excel(outpath, sheet_name='MergedDataset', index=False)